# Application d'analyse des données de Formule 1 saison 2022
celle que je peux récupérer en tout cas

Cette application est réutilisable pour les prochaines saisons

## Imports

In [6]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json

## Backend
dans cette partie, je fais tous ce qui ne relève pas de l'affichage des graphs.
Cela permet d'avoir directement toutes les données d'un coup pour ensuite juste avoir a
choisir le graph a afficher

In [13]:
# import data
data_result = pd.read_csv('data/race_result.csv', sep=",")
data_points = pd.read_csv('data/race_points.csv', sep=",")
jsonfile = open("teams.json")
teams_pilot = json.load(jsonfile)

## Frontend
Ici les affichages seront séparé pour pouvoir choisir ce qu'on affiche

### Test data import

In [14]:
print(data_result)
print(data_points)
print(teams_pilot)

Empty DataFrame
Columns: [location, date, ham, rus, ver, per, lec, sai, gas, tsu, ric, nor, oco, alo, vet, str, lat, alb, bot, zho, mag, sch]
Index: []

[0 rows x 22 columns]
Empty DataFrame
Columns: [location, date, ham, rus, ver, per, lec, sai, gas, tsu, ric, nor, oco, alo, vet, str, lat, alb, bot, zho, mag, sch]
Index: []

[0 rows x 22 columns]
{'redbull': ['ver', 'per'], 'mrecedes': ['ham', 'rus'], 'ferrari': ['lec', 'sai'], 'mclaren': ['ric', 'nor'], 'alpine': ['alo', 'oco'], 'alphatauri': ['gas', 'tsu'], 'astonmartin': ['vet', 'str'], 'williams': ['lat', 'alb'], 'alfaromeo': ['bot', 'zho'], 'haas': ['mag', 'sch']}
